# 01 — Funções e comprehensions (focado em dados)

Objetivo: revisar padrões de Python que você usa o tempo todo em limpeza e preparação de dados.

Tempo: ~20 min

## Setup

Este notebook usa um dataset pequeno em `dados/sample/` para você rodar tudo localmente.

In [ ]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até encontrar uma 'marca' do repositório (README.md + pasta data)."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    # fallback: assume cwd
    return Path.cwd().resolve()

ROOT = find_repo_root()
DATA_DIR = ROOT / "data"
SAMPLE_DIR = DATA_DIR / "sample"

print("ROOT:", ROOT)
print("SAMPLE_DIR:", SAMPLE_DIR)

In [ ]:
import pandas as pd

sales = pd.read_csv(SAMPLE_DIR / "sales.csv")
customers = pd.read_csv(SAMPLE_DIR / "customers.csv")

display(sales.head())
display(customers.head())

## 1) Comprehensions (list/dict)

Use para transformar colunas, filtrar registros e criar estruturas auxiliares.

In [ ]:
# Vamos criar um "dataframe" leve como lista de dicionários
rows = sales.to_dict(orient="records")

# Exemplo 1: filtrar vendas com receita > 300
high = [r for r in rows if r["revenue"] > 300]

# Exemplo 2: criar lista de tuplas (order_id, revenue)
order_revenue = [(r["order_id"], r["revenue"]) for r in rows]

# Exemplo 3: dicionário de receita por order_id
revenue_by_order = {r["order_id"]: r["revenue"] for r in rows}

len(high), order_revenue[:3], list(revenue_by_order.items())[:3]

## 2) Funções pequenas e reutilizáveis

Regra prática para dados: escreva funções curtas que façam **uma coisa** e sejam fáceis de testar.

In [ ]:
from typing import Iterable

def safe_float(x) -> float | None:
    """Tenta converter para float; devolve None se falhar (útil em parsing de strings)."""
    try:
        return float(x)
    except (TypeError, ValueError):
        return None

def top_n(values: Iterable[float], n: int = 5) -> list[float]:
    """Retorna os N maiores valores."""
    return sorted(values, reverse=True)[:n]

# Exemplo rápido com receitas
top_n(sales["revenue"].tolist(), 5)

## 3) Aplicando no Pandas (com `.apply` e com vetorização)

Em dados, prefira vetorização sempre que possível — mas `.apply` resolve rápido em casos específicos.

In [ ]:
import numpy as np

# Exemplo: classificar ticket médio (revenue) em faixas
def bucket_revenue(v: float) -> str:
    if v < 150:
        return "baixo"
    if v < 300:
        return "medio"
    return "alto"

sales["ticket_faixa_apply"] = sales["revenue"].apply(bucket_revenue)

# Alternativa vetorizada com numpy.select
conds = [sales["revenue"] < 150, sales["revenue"].between(150, 299.99)]
choices = ["baixo", "medio"]
sales["ticket_faixa_vec"] = np.select(conds, choices, default="alto")

sales[["revenue","ticket_faixa_apply","ticket_faixa_vec"]].head()

## Exercícios

1- Crie uma coluna `is_weekend` (True/False) a partir de `date`.
2- Crie uma função que normalize nomes de produto (ex.: `strip`, `lower`).
3- Gere um dicionário `receita_por_categoria` usando comprehension (sem usar pandas).

In [ ]:
# Escreva suas respostas aqui
